# P_Tuning微调

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForCausalLM,Trainer,TrainingArguments,DataCollatorForSeq2Seq
from peft import PromptEncoderReparameterizationType,PromptEncoderConfig,get_peft_model,TaskType
import warnings
warnings.filterwarnings("ignore")

E:\web\LLM-tuning\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\web\LLM-tuning\.venv\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## 1、加载模型和分词器

In [2]:
model_name="Qwen/Qwen3-0.6B"
model=AutoModelForCausalLM.from_pretrained(model_name,device_map='auto',torch_dtype="auto",)
tokenizer=AutoTokenizer.from_pretrained(model_name)

Loading weights: 100%|██████████| 311/311 [00:00<00:00, 420.63it/s, Materializing param=model.norm.weight]                              
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


## 2、PromptEncoderConfig配置

In [3]:
config = PromptEncoderConfig(
    task_type=TaskType.CAUSAL_LM, 
    num_virtual_tokens=10,
    encoder_reparameterization_type=PromptEncoderReparameterizationType.LSTM,
    encoder_dropout=0.2
)
model=get_peft_model(model=model,peft_config=config)
model.print_trainable_parameters()

trainable params: 48,280,576 || all params: 799,912,960 || trainable%: 6.0357


## 3、加载数据集

In [4]:
dataset = load_dataset("Kuugo/chinese_law_ft_dataset", split="train[:1000]",cache_dir='../../datasets').train_test_split(train_size=0.7,shuffle=True,seed=7)

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'id'],
        num_rows: 700
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'id'],
        num_rows: 300
    })
})

In [7]:
dataset['train']

Dataset({
    features: ['instruction', 'input', 'output', 'id'],
    num_rows: 700
})

## 4、数据预处理：分词、编码

In [8]:
def process_fun(example):
    content=[]
    for instruction,input,output in zip(example['instruction'],example['input'],example['output']):
        if input.strip():
            text=f'Human:{instruction}\n{input}\nAI:{output}'
            content.append(text)
        else:
            text=f'Human:{instruction}\nAI:{output}'
            content.append(text)
    
    encoded = tokenizer(
        content,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    labels = encoded["input_ids"].clone()
    for index,text in enumerate(content):
        answer_start=text.find('AI:')+len('AI:')
        question=text[:answer_start]
        question_ids=tokenizer.encode(question, add_special_tokens=False)
        question_length=len(question_ids)
        labels[index,:question_length]=-100
    return {
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"],
        "labels": labels
    }


In [9]:
train_process_data=dataset['train'].map(process_fun,batched=True,remove_columns=dataset['train'].column_names)
test_process_data=dataset['test'].map(process_fun,batched=True,remove_columns=dataset['test'].column_names)

In [10]:
tokenizer.decode(train_process_data[1],skip_special_tokens=True)

'Human:某公司计划与一家同行公司合并，合并后的权利和义务如何分配？\nAI:根据民法典总则第六十七条规定，法人合并的，其权利和义务由合并后的法人享有和承担。因此，在合并后，原公司和同行公司的权利和义务将合并在一起，并由合并后的新公司享有和承担。'

## 5、模型训练配置

In [11]:
# 定义训练参数
training_args = TrainingArguments(
    output_dir="../../models/p_tuning",
    logging_steps=10,
    logging_dir='./runs',
    eval_strategy='epoch',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-4,
    weight_decay=0.01,
    metric_for_best_model='eval_loss',
    gradient_accumulation_steps=4,
    fp16=True
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [12]:
trainer=Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_process_data,
    train_dataset=train_process_data,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True),
)

## 6、模型训练

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.528687,0.470597
2,0.419307,0.406705
3,0.425215,0.401204


TrainOutput(global_step=132, training_loss=1.5604548878741986, metrics={'train_runtime': 5293.6793, 'train_samples_per_second': 0.397, 'train_steps_per_second': 0.025, 'total_flos': 3845237243904000.0, 'train_loss': 1.5604548878741986, 'epoch': 3.0})

## 7、模型训练

In [14]:
trainer.save_model('../../models/p_tuning')